# 1.3) Supervised Localization

An EfficientNet is trained here to perform a regression task which assigns a location, that is the x and y position, to each laser point in an image.

# Import statements
Following packages are necessary:

In [1]:
import random
import utils

import tensorflow as tf
import numpy as np

from efficientnet.tfkeras import EfficientNetB0

#import efficientnet
#from efficientnet.tfkeras import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from lastengen import LASTENSequence
from matplotlib import pyplot as plt

# Checks

The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.0.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Data

Import training images

In [3]:
path_train = r'D:\LASTEN\train'
path_validation = r'D:\LASTEN\validation'
path_test = r'D:\LASTEN\test'

#path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/train'
#path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
#path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'

width = 384
height = 384

grid_width = 18
grid_height = 18

# Seeding

Seeds are set to ensure reproducible training.

In [4]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network

An EfficientNet network is instantiated with keras to run a regression task.

In [5]:
base_model = EfficientNetB0(weights=None, include_top=False, input_shape=(height, width, 1))

x = base_model.output
x = GlobalAveragePooling2D()(x)

predictions = Dense(2 * grid_width * grid_height, activation='linear',)(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 384, 1) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 192, 192, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 192, 192, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 192, 192, 32) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

# Data Preparation

The generator sequence for training is built.

In [6]:
image_ids = None

train_gen = LASTENSequence(path_train,
                           batch_size=16,
                           image_ids=image_ids,
                           #preprocess_input=preprocess_input,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height,
                           label="keypoints")

validation_gen = LASTENSequence(path_validation,
                                batch_size=16,
                                #preprocess_input=preprocess_input,
                                width=width,
                                height=height,
                                label="keypoints")

#test_gen = LASTENSequence(path_train,
#                          batch_size=8,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

# Training Preparation

Prepare the settings for training the model.

In [7]:
save_path = "experiments/1_3_localization"

logger = CSVLogger(save_path + "/result.log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5",
                          period=10)

callbacks = [logger, checker]

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Training

The model is fitted with a generator.

In [ ]:
model.fit_generator(train_gen,
                    epochs=300,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

Epoch 1/300
11/11 [==============================] - 36s 3s/step - loss: 36804.7212 - mae: 177.4322 - val_loss: 38321.6667 - val_mae: 182.6386
Epoch 2/300
11/11 [==============================] - 23s 2s/step - loss: 35296.7931 - mae: 173.8348 - val_loss: 26667.0267 - val_mae: 150.8605
Epoch 3/300
11/11 [==============================] - 23s 2s/step - loss: 33925.3349 - mae: 170.2818 - val_loss: 23004.7897 - val_mae: 139.2756
Epoch 4/300
10/11 [==========================>...] - ETA: 0s - loss: 32627.3373 - mae: 166.7602

In [ ]:
X, y = train_gen[0]
y_pred = model.predict(X)

In [ ]:
plt.rcParams['figure.figsize'] = (15, 15)

In [ ]:
image = X[0]
keypoints = y_pred[0]

plt.clf()
plt.imshow(image[:,:,0], cmap="gray")

x_list = []
y_list = []

for index in range(0, 18*18):
    x_index = index * 2 
    
    x = (keypoints[x_index] - 1) * width
    y = (keypoints[x_index + 1] - 1) * height
    
    x_list.append(x)
    y_list.append(y)
    
    
plt.scatter(x_list, y_list,s=80,facecolors='none', edgecolors='r')